In [1]:
# Importing all the neede 
import os
import itertools
import numpy as np

#from Bio.Blast import NCBIXML

# Biopython is required for reading multifasta files and storing sequences.
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Alphabet import IUPAC

from Utils import *

# Poner condiciones sobre el tamano de la subunidad

In [2]:
subunit_16s = 1542
subunit_23s = 2905
subunit_5s = 120

In [3]:
# Other things
mainPath = "/Users/CamilaMV/Desktop/sortmernaFiles/Genos_8000_800/"
mainPath2 = "/Users/CamilaMV/Desktop/sortmernaFiles/Genos_8000_800/FastaFiles/"

In [4]:
#mainPath = "/Users/CamilaMV/Desktop/sortmernaFiles/Genos_3000_300/"
#mainPath2 = "/Users/CamilaMV/Desktop/sortmernaFiles/Genos_3000_300/FastaFiles/"

In [5]:
def parse_tabular_blast_results(blast_results_file):
    """
    Parse out the BLAST results from a tabular format BLAST results file into a 
    2D dictionary containing query id/name -> subject id/name -> BLAST result.
    Args:
        blast_results_file (string): Path to tabular BLAST results file.
    Returns:
        2D dictionary containing query id/name -> subject id/name -> BLAST result
    """
    # results are stored in a 2D dictionary - query -> subject -> BLAST result
    results = {}
    with open(blast_results_file,'r') as f:
        for line in f:
            line = line.rstrip()
            sp = line.split('\t')
            # get query id
            q = sp[0]
            q_len = query_lengths[q]
            # get subject id
            s = sp[1]
            # get % identity and convert to decimal (e.g. 96.7 -> 0.967)
            pid1 = float(sp[2])
            pid2 = float(sp[2])/100.0
            # get alignment length
            aln_len = int(sp[3])
            # get query start index
            q_s_i = int(sp[6])
            # get query end index
            q_e_i = int(sp[7])
            # get subject start index
            s_s_i = int(sp[8])
            # get subject end index
            s_e_i = int(sp[9])
            
            q_cov = sp[12]
            
            # number of identities (count of positive sequence matches)
            identities = int(aln_len*pid2)
            
            # the results for a particular query could be whatever you need
            # in this example, a dictionary is made where different results have
            # different identifiers for instance
            result_dict = {'aln_len':aln_len, 
            'q_len':q_len,
            'pid':pid1,
            'identities':identities,
            'q_s_i':q_s_i, 
            'q_e_i':q_e_i,
            's_s_i':s_s_i,
            's_e_i':s_e_i,
            'qcov':q_cov
            }
            
            # could add more items to this BLAST result dictionary
            if q not in results:
                results[q] = {s:[result_dict]}
            else:
                if s not in results[q]:
                    results[q][s] = [result_dict]
                else:
                    results[q][s].append(result_dict)
            
    return results
    

# Reading blast files for 16s, 23s and 5s for all genos

In [6]:
# getting blast genos
folder1 = mainPath +"Results16s/"
#genosNames = Processing_files_from_folder(folder1,10)

In [7]:
print os.path.isdir(mainPath)

True


In [8]:
print mainPath, mainPath2

/Users/CamilaMV/Desktop/sortmernaFiles/Genos_8000_800/ /Users/CamilaMV/Desktop/sortmernaFiles/Genos_8000_800/FastaFiles/


In [9]:
genosNamesFasta,genosNamesFastaExists = Processing_files_from_folder(mainPath2)

In [10]:
#genosNamesFinalBlast = Getting_files_path_by_extension(genosNames,".blast")[0]

In [11]:
#genosNamesFinalNames = Getting_files_path_by_extension(genosNamesFinal,".split")[0]
nameGenos = []
for genoName in genosNamesFasta:
    
    nameGenos.append(genoName.split(".split")[0])

In [12]:
print genosNamesFasta

print ()

print nameGenos

['2362_nt.split.8Kmer.800overlap.fasta.fastaN', 'B1-CDA_nt.split.8Kmer.800overlap.fasta.fastaN', 'C3-41_nt.split.8Kmer.800overlap.fasta.fastaN', 'CBAM5_nt.split.8Kmer.800overlap.fasta.fastaN', 'III37_nt.split.8Kmer.800overlap.fasta.fastaN', 'Ot4b.25_nt.split.8Kmer.800overlap.fasta.fastaN', 'Ot4b.31_nt.split.8Kmer.800overlap.fasta.fastaN', 'Ot4b.49_nt.split.8Kmer.800overlap.fasta.fastaN', 'RB-21.split.8Kmer.800overlap.fasta.fastaN', 'Silvestris.split.8Kmer.800overlap.fasta.fastaN']
()
['2362_nt', 'B1-CDA_nt', 'C3-41_nt', 'CBAM5_nt', 'III37_nt', 'Ot4b.25_nt', 'Ot4b.31_nt', 'Ot4b.49_nt', 'RB-21', 'Silvestris']


In [13]:
folder16s = mainPath+"Results16s/"
folder23s = mainPath+"Results23s/"
folder5s = mainPath+"Results5s/"

In [14]:
print folder16s
print folder23s
print folder5s

/Users/CamilaMV/Desktop/sortmernaFiles/Genos_8000_800/Results16s/
/Users/CamilaMV/Desktop/sortmernaFiles/Genos_8000_800/Results23s/
/Users/CamilaMV/Desktop/sortmernaFiles/Genos_8000_800/Results5s/


# Processing blast results of 16s, 23s and 5s for all genos

In [15]:
os.getcwd()

'/Users/CamilaMV/Desktop/TESIS/Python_Notebooks/Tesis_2016_2'

In [16]:
os.chdir('/Users/CamilaMV/Desktop/')

In [17]:
os.getcwd()

'/Users/CamilaMV/Desktop'

# Getting the Operons subunits for each geno

In [18]:
#path16s, path23s and path5s are constant
genos_results_subunits_dict = {}

# Processing tabular results
for genoFasta, gname in itertools.izip(genosNamesFasta,nameGenos):
    
    # Query lengths temporal
    pathTemp = mainPath2 + genoFasta
    query_files = [pathTemp]
    query_lengths = {rec.name:len(rec.seq) for q_file in query_files for rec in SeqIO.parse(q_file, 'fasta')}
        
    # Processing blast results
    p16s = folder16s+genoFasta+".16s.blast"
    p23s = folder23s+genoFasta+".23s.blast"
    p5s = folder5s+genoFasta+".5s.blast"
        
    # Using function parse_tabular_blast_results, NOT MINE
    results_geno_temp_16s = parse_tabular_blast_results(p16s)
    results_geno_temp_23s = parse_tabular_blast_results(p23s)
    results_geno_temp_5s = parse_tabular_blast_results(p5s)
    
    # Making intersection for each geno
    intersection_16_23s_geno_temp = set(results_geno_temp_16s.keys()).intersection(results_geno_temp_23s.keys())
    intersection_16_23_5s_geno_temp = intersection_16_23s_geno_temp.intersection(results_geno_temp_5s.keys())
    
    # Putting results for each geno as value of dictionary
    dict_frag_subunits = {}
    for keyFrag in intersection_16_23_5s_geno_temp:
        
        listTemp = [results_geno_temp_16s[keyFrag],results_geno_temp_23s[keyFrag],results_geno_temp_5s[keyFrag]]
        
        dict_frag_subunits[keyFrag] = listTemp
        
    # Putting final results in big dictionary
    genos_results_subunits_dict[gname] = dict_frag_subunits 

In [19]:
genos_results_subunits_dict.keys()

['CBAM5_nt',
 '2362_nt',
 'C3-41_nt',
 'Ot4b.25_nt',
 'Silvestris',
 'B1-CDA_nt',
 'Ot4b.31_nt',
 'RB-21',
 'Ot4b.49_nt',
 'III37_nt']

# Looking the results

In [20]:
genos_results_subunits_dict

{'2362_nt': {'Frag_393': [{'MedSativ': [{'aln_len': 93,
      'identities': 79,
      'pid': 85.7,
      'q_e_i': 6440,
      'q_len': 8000,
      'q_s_i': 6348,
      'qcov': '1.16',
      's_e_i': 1658,
      's_s_i': 1562}]},
   {'CloTet37': [{'aln_len': 158,
      'identities': 123,
      'pid': 78.4,
      'q_e_i': 6426,
      'q_len': 8000,
      'q_s_i': 6269,
      'qcov': '1.98',
      's_e_i': 281,
      's_s_i': 113}]},
   {'CP000817.1/649536-649653': [{'aln_len': 116,
      'identities': 114,
      'pid': 98.3,
      'q_e_i': 4817,
      'q_len': 8000,
      'q_s_i': 4702,
      'qcov': '1.45',
      's_e_i': 118,
      's_s_i': 1}]}],
  'Frag_394': [{'LsnCreso': [{'aln_len': 896,
      'identities': 887,
      'pid': 99.1,
      'q_e_i': 901,
      'q_len': 8000,
      'q_s_i': 6,
      'qcov': '11.2',
      's_e_i': 1444,
      's_s_i': 549}]},
   {'Bac12248': [{'aln_len': 2931,
      'identities': 2928,
      'pid': 99.9,
      'q_e_i': 4195,
      'q_len': 8000,
      '

In [21]:
#listTest = sorted(genos_results_subunits_dict['2362_nt'].keys())
#fragNumbersTest = [ fragTemp.split("_")[1] for fragTemp in listTest] 

In [22]:
#fragNumbersTest

In [23]:
#genos_results_subunits_dict['2362_nt']['Frag_1099']

# Making dataframe of fragments of all genomes

In [24]:
# Making dataframe of fragments of all genomes
#genos_results_subunits_dict

In [25]:
for key in genos_results_subunits_dict:
    
    print key
    
    for key2 in genos_results_subunits_dict[key]:
        print key2
        listKey2 = genos_results_subunits_dict[key][key2]
    
        listInsideTempAln_Len = []
        for dictTemp in listKey2:
            for insideKey in dictTemp:
                listInsideTempAln_Len.append(dictTemp[insideKey][0]['aln_len'])
            
        print listInsideTempAln_Len
        
        if(listInsideTempAln_Len[0]>=1300):
            print "Subunit accepted"
        if(listInsideTempAln_Len[1]>=2460):
            print "Subunit accepted"
        if(listInsideTempAln_Len[2]>=100):
            print "Subunit accepted"
    
    print "\n"

CBAM5_nt
Frag_740
[165, 1515, 116]
Subunit accepted
Frag_486
[29, 57, 116]
Subunit accepted
Frag_583
[172, 162, 62]
Frag_733
[74, 76, 116]
Subunit accepted
Frag_732
[69, 69, 44]
Frag_523
[1363, 2931, 116]
Subunit accepted
Subunit accepted
Subunit accepted
Frag_648
[84, 101, 116]
Subunit accepted


2362_nt
Frag_434
[1363, 2931, 116]
Subunit accepted
Subunit accepted
Subunit accepted
Frag_504
[75, 779, 116]
Subunit accepted
Frag_566
[1363, 2931, 116]
Subunit accepted
Subunit accepted
Subunit accepted
Frag_394
[896, 2931, 116]
Subunit accepted
Subunit accepted
Frag_550
[1363, 2931, 116]
Subunit accepted
Subunit accepted
Subunit accepted
Frag_453
[1363, 2931, 116]
Subunit accepted
Subunit accepted
Subunit accepted
Frag_393
[93, 158, 116]
Subunit accepted
Frag_591
[1363, 2931, 116]
Subunit accepted
Subunit accepted
Subunit accepted
Frag_541
[1363, 2931, 116]
Subunit accepted
Subunit accepted
Subunit accepted
Frag_535
[1363, 402, 44]
Subunit accepted
Frag_536
[1363, 2931, 116]
Subunit accept

In [26]:
def making_df_by_geno(genoName,frag_temp_by_geno,aln_by_frag_temp_by_geno):
    
      # importing pandas
    import pandas as pd
    from pandas import DataFrame
    
    len1 = len(frag_temp_by_geno)
    len2 = len(aln_by_frag_temp_by_geno)
    
    if(len1 != len2):
        
        print "There is a problem"
    else:
        dColumns={
            '1_GenoName' : genoName,
            '2_FragNumbers' : frag_temp_by_geno,
            '3_Aln_sizes': aln_by_frag_temp_by_geno
        }    
    
    df = pd.DataFrame(dColumns)
    
    return(df)

In [27]:
## Function 25
def parsing_data_as_df_for_each_geno(genos_results_subunits_dict):
    
    # importing pandas
    import pandas as pd
    from pandas import DataFrame

    # Dict genos
    
    dict_genos_lists_for_df = {}
    
    frag_temp_by_geno = []
    aln_by_frag_temp_by_geno = []
    
    for key in genos_results_subunits_dict:
        # Geno key
        print key
    
        for key2 in genos_results_subunits_dict[key]:
            
            # Frag key
            #print key2
            listKey2 = genos_results_subunits_dict[key][key2]
            
            listInsideTempAln_Len = []
            for dictTemp in listKey2:
                for insideKey in dictTemp:
                    listInsideTempAln_Len.append(dictTemp[insideKey][0]['aln_len'])
                
            # List of aln 
            if(listInsideTempAln_Len[0]>=1300 and listInsideTempAln_Len[1]>=2460 and listInsideTempAln_Len[2]>=100):
                #print "Operons accepted"
                print key2
                print listInsideTempAln_Len
                # filling frag list
                frag_temp_by_geno.append(key2)
                aln_by_frag_temp_by_geno.append(listInsideTempAln_Len)
                
        print "\n"       
        
        dfTemp = making_df_by_geno(key,frag_temp_by_geno,aln_by_frag_temp_by_geno)
        dict_genos_lists_for_df[key]= dfTemp

    return(dict_genos_lists_for_df)

In [28]:
# parsing as dataframe

r1test = parsing_data_as_df_for_each_geno(genos_results_subunits_dict)

CBAM5_nt
Frag_523
[1363, 2931, 116]


2362_nt
Frag_434
[1363, 2931, 116]
Frag_566
[1363, 2931, 116]
Frag_550
[1363, 2931, 116]
Frag_453
[1363, 2931, 116]
Frag_591
[1363, 2931, 116]
Frag_541
[1363, 2931, 116]
Frag_536
[1363, 2931, 116]


C3-41_nt
Frag_106
[1362, 2929, 116]
Frag_16
[1362, 2930, 116]
Frag_616
[1363, 2931, 116]
Frag_3
[1363, 2931, 116]


Ot4b.25_nt
Frag_121
[1363, 2930, 116]
Frag_157
[1363, 2931, 116]
Frag_90
[1363, 2931, 116]
Frag_76
[1363, 2931, 116]
Frag_89
[1363, 2930, 115]


Silvestris
Frag_435
[1405, 2926, 116]
Frag_471
[1405, 2926, 116]
Frag_547
[1405, 2926, 116]
Frag_534
[1364, 2926, 116]
Frag_533
[1405, 2926, 116]
Frag_464
[1405, 2926, 116]
Frag_524
[1405, 2926, 116]
Frag_525
[1405, 2926, 116]


B1-CDA_nt


Ot4b.31_nt


RB-21
Frag_114
[1363, 2931, 116]
Frag_639
[1363, 2931, 116]
Frag_2
[1363, 2931, 116]
Frag_667
[1363, 2931, 116]
Frag_658
[1363, 2931, 116]
Frag_1
[1363, 2930, 116]
Frag_157
[1363, 2931, 116]
Frag_77
[1363, 2931, 116]


Ot4b.49_nt
Frag_549
[1363, 29

In [29]:
r1test

{'2362_nt':   1_GenoName 2_FragNumbers        3_Aln_sizes
 0    2362_nt      Frag_523  [1363, 2931, 116]
 1    2362_nt      Frag_434  [1363, 2931, 116]
 2    2362_nt      Frag_566  [1363, 2931, 116]
 3    2362_nt      Frag_550  [1363, 2931, 116]
 4    2362_nt      Frag_453  [1363, 2931, 116]
 5    2362_nt      Frag_591  [1363, 2931, 116]
 6    2362_nt      Frag_541  [1363, 2931, 116]
 7    2362_nt      Frag_536  [1363, 2931, 116],
 'B1-CDA_nt':    1_GenoName 2_FragNumbers        3_Aln_sizes
 0   B1-CDA_nt      Frag_523  [1363, 2931, 116]
 1   B1-CDA_nt      Frag_434  [1363, 2931, 116]
 2   B1-CDA_nt      Frag_566  [1363, 2931, 116]
 3   B1-CDA_nt      Frag_550  [1363, 2931, 116]
 4   B1-CDA_nt      Frag_453  [1363, 2931, 116]
 5   B1-CDA_nt      Frag_591  [1363, 2931, 116]
 6   B1-CDA_nt      Frag_541  [1363, 2931, 116]
 7   B1-CDA_nt      Frag_536  [1363, 2931, 116]
 8   B1-CDA_nt      Frag_106  [1362, 2929, 116]
 9   B1-CDA_nt       Frag_16  [1362, 2930, 116]
 10  B1-CDA_nt      Frag

In [30]:
def complete_list_dfs(dict_test_list_dfs):
    
    all_dfs = []
    for key in dict_test_list_dfs:
        
        dfTemp = dict_test_list_dfs[key]
        all_dfs.append(dfTemp)
        
    return all_dfs

In [31]:
r1test.keys()

['CBAM5_nt',
 '2362_nt',
 'C3-41_nt',
 'Ot4b.25_nt',
 'Silvestris',
 'III37_nt',
 'Ot4b.31_nt',
 'RB-21',
 'Ot4b.49_nt',
 'B1-CDA_nt']

In [32]:
os.getcwd()

'/Users/CamilaMV/Desktop'

In [33]:
# Using complete_list_dfs to get list of dfs
all_dfs_with_last_properties = complete_list_dfs(r1test)

os.getcwd()

# run function
multiple_dfs(all_dfs_with_last_properties, '10_Geno_Jeny', '10_Proca_8k-800.xlsx', 1)


In [34]:
#listInsideTempAln_Len = []
#for dictTemp in listKey2:
    
#    for insideKey in dictTemp:
        
#        listInsideTempAln_Len.append(dictTemp[insideKey][0]['aln_len'])

# It does not work as function ??, I have to know why ?

In [35]:
# It does not work as function ??, I have to know why ?

"""
def processing_subunit_blast_files_for_genos(genosNamesFasta,genosNamesFinalBlast,nameGenos,folder16s,folder23s,folder5s,mainPath2):
    
    #path16s, path23s and path5s are constant
    genos_results_subunits_dict = {}
    
    # Processing tabular results
    for genoFasta, genoblast, gname in itertools.izip(genosNamesFasta,genosNamesFinalBlast,nameGenos):
        
        # Query lengths temporal
        pathTemp = mainPath2 + genoFasta
        query_files = [pathTemp]
        query_lengths = {rec.name:len(rec.seq) for q_file in query_files for rec in SeqIO.parse(q_file, 'fasta')}
        
        # Processing blast results
        p16s = folder16s+genoblast
        p23s = folder23s+genoblast
        p5s = folder5s+genoblast
        
        # Using function parse_tabular_blast_results, NOT MINE
        results_geno_temp_16s = parse_tabular_blast_results(p16s)
        results_geno_temp_23s = parse_tabular_blast_results(p23s)
        results_geno_temp_5s = parse_tabular_blast_results(p5s)
    
        # Making intersection for each geno
        intersection_16_23s_geno_temp = set(results_geno_temp_16s.keys()).intersection(results_geno_temp_23s.keys())
        intersection_16_23_5s_geno_temp = intersection_16_23s_geno_temp.intersection(results_geno_temp_5s.keys())
    
        # Putting results for each geno as value of dictionary
        dict_frag_subunits = {}
        for keyFrag in intersection_16_23_5s_geno_temp:
        
            listTemp = [results_geno_temp_16s[keyFrag],results_geno_temp_23s[keyFrag],results_geno_temp_5s[keyFrag]]
        
            dict_frag_subunits[keyFrag] = listTemp
        
        # Putting final results in big dictionary
        genos_results_subunits_dict[gname] = dict_frag_subunits 
    
    return genos_results_subunits_dict
    
"""

"\ndef processing_subunit_blast_files_for_genos(genosNamesFasta,genosNamesFinalBlast,nameGenos,folder16s,folder23s,folder5s,mainPath2):\n    \n    #path16s, path23s and path5s are constant\n    genos_results_subunits_dict = {}\n    \n    # Processing tabular results\n    for genoFasta, genoblast, gname in itertools.izip(genosNamesFasta,genosNamesFinalBlast,nameGenos):\n        \n        # Query lengths temporal\n        pathTemp = mainPath2 + genoFasta\n        query_files = [pathTemp]\n        query_lengths = {rec.name:len(rec.seq) for q_file in query_files for rec in SeqIO.parse(q_file, 'fasta')}\n        \n        # Processing blast results\n        p16s = folder16s+genoblast\n        p23s = folder23s+genoblast\n        p5s = folder5s+genoblast\n        \n        # Using function parse_tabular_blast_results, NOT MINE\n        results_geno_temp_16s = parse_tabular_blast_results(p16s)\n        results_geno_temp_23s = parse_tabular_blast_results(p23s)\n        results_geno_temp_5s = pa

In [36]:
#testBigDict = processing_subunit_blast_files_for_genos(genosNamesFasta,genosNamesFinalBlast,nameGenos,folder16s,folder23s,folder5s,mainPath2)

In [37]:
#for root, dirs, files in os.walk("/Users/CamilaMV/Desktop/", topdown=False):
#    for name in files:
#        print(os.path.join(root, name))
#    for name in dirs:
#        print(os.path.join(root, name))